In [10]:
import matplotlib
import numpy as np
import os
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import datetime

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):
        config = compose(config_name='config_hlwas_coadd.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):
        config = compose(config_name='config_hlwas_coadd.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.utils import util

survey_params = util.hydra_to_dict(config.survey)

In [11]:
pipeline_params = util.hydra_to_dict(config.pipeline)
debugging = True

if debugging:
    pipeline_dir = f'{config.machine.pipeline_dir}_coadd'

print(f'pipeline_dir: {pipeline_dir}')

pipeline_dir: /data/bwedig/mejiro/pipeline_coadd


# Detectable and candidate lens counts

Number of detectable strong lenses per square degree of simulated area, candidate strong lenses

In [12]:
# combine CSVs from each run into one CSV
data_dir = os.path.join(pipeline_dir, '00')

detectable_csv = os.path.join(data_dir, 'detectable_pop.csv')
util.delete_if_exists(detectable_csv)
detectable_df = util.combine_all_csvs(data_dir, 'detectable', detectable_csv)

if survey_params['total_population']:
    total_csv = os.path.join(data_dir, 'total_pop.csv')
    util.delete_if_exists(total_csv)
    total_df = util.combine_all_csvs(data_dir, 'total', total_csv)

Wrote combined CSV to /data/bwedig/mejiro/pipeline_coadd/00/detectable_pop.csv
Wrote combined CSV to /data/bwedig/mejiro/pipeline_coadd/00/total_pop.csv


In [13]:
survey_area = survey_params['area']
runs = survey_params['runs']

total_area = survey_area * runs
print(f'{runs} run(s) of {survey_area} sq. deg. each gives {total_area:.2f} sq. deg. total')

54 run(s) of 0.5 sq. deg. each gives 27.00 sq. deg. total


In [14]:
print(f'{len(detectable_df)} detectable strong lenses')

if survey_params['total_population']:
    print(f'{len(total_df)} total candidate strong lenses')
    fraction_detectable = len(detectable_df) / len(total_df)
    print(f'Fraction of candidate strong lenses that are detectable: {fraction_detectable:.5f}')

4693 detectable strong lenses
460562 total candidate strong lenses
Fraction of candidate strong lenses that are detectable: 0.01019


In [15]:
det_per_sq_deg = len(detectable_df) / total_area
print(f'Detectable strong lenses per sq. deg.: {det_per_sq_deg:.2f}')

if survey_params['total_population']:
    total_per_sq_deg = len(total_df) / total_area
    print(f'Candidate strong lenses per sq. deg.: {round(total_per_sq_deg)}')

Detectable strong lenses per sq. deg.: 173.81
Candidate strong lenses per sq. deg.: 17058


In [16]:
det_per_exposure = det_per_sq_deg * 0.281
print(f'Detectable strong lenses per exposure: {det_per_exposure:.2f}')

Detectable strong lenses per exposure: 48.84


In [17]:
snr_threshold = 200
high_snr_df = detectable_df[detectable_df['snr'] > snr_threshold]
high_snr_det_per_sq_deg = len(high_snr_df) / total_area
high_snr_det_per_exposure = high_snr_det_per_sq_deg * 0.281

print(f'{len(high_snr_df)} characterizable strong lenses (SNR > {snr_threshold})')
print(f'{high_snr_det_per_sq_deg:.2f} characterizable per sq. deg.')
print(f'Characterizable strong lenses per exposure: {high_snr_det_per_exposure:.2f}')

84 characterizable strong lenses (SNR > 200)
3.11 characterizable per sq. deg.
Characterizable strong lenses per exposure: 0.87


In [18]:
# HLWAS
print(f'{round(det_per_sq_deg * 1700)} detectable strong lenses in HLWAS')
print(f'{round(high_snr_det_per_sq_deg * 1700)} characterizable strong lenses in HLWAS')

295485 detectable strong lenses in HLWAS
5289 characterizable strong lenses in HLWAS
